# Vehicular interaction and movement for simple traffic configurations
- We are interested in the dynamics between different types of vehicles on a grid. 
- Particularly, we would like to investigate the interaction between the speeds and general movement of cars (fast vehicles) and bicyles (slower vehicles). 
- Our research question: __can increased volume of bicycles (slower vehicles) in a traffic system increase its total throughput?__

# Model design
Here we describe the design of our system and the assumptions made on its outset. In general, our model contains both deterministic rules and stochastic processes.

### Assumptions
Our model specific assumptions are as follows:
1. There are no collisions. We expect drivers to be responsible and always make choices to slow down and wait their turn at intersections.
2. Destination is irrelevant. We are not concerned with where vehicles start/exit the system. Hence, decisions with regard to how a vehicles move at junction points are handled with randomness. 
3. Cells can take on 5 separate states:
 - Car $(C)$: A fast vehicle that moves each timestep if the rules allow it.
 - Bicycle $(B)$: A slow vehicle that moves every $b$ timesteps relative to the time it entered the system.
 - Empty $(E)$: Grass/division between roads. Cells marked $(E)$ stay that way for the entire simulation.
 - Road $(R)$: An empty cell that a vehicle can occupy.
 - Intersection $(I)$: A 4-way stop sign at which vehicles must stop. Intersections act in a first in, first out manner meaning each vehicle must wait its turn to move. Bicycles $(B)$ may move at its turn regardless of whether or not the it is a $b$th timestep.
4. Passing is allowable. Cars may pass bicycles if they have a specific neighborhood, described below.


### Rules
___
Our rules are fairly involved. This is because vehicles can pass eachother. In addition, there is a special case for intersection nodes on the system.
#### These are the basic rules for vehicles $V$ moving forward
<img src=presfigs/rules.png>
#### These are the dynamics associated with cars passing bicycles
<img src=presfigs/passing_rules.png>


In [12]:
# necessary imports
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import colors
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

import sys, os
import imageio
from glob import glob

from traffic import *

# Model Parameters and Set Up:

Parameters we want to __set__ for a class of runs:
   * time - total time to run sys fortime
   * mu - time delay for bicycles
   * dim - dimensions of the system grid
   
Our main arguments to __change__ (testing parameters):
   * flows - flow rates from each direction
       *     these are just the probability of a vehicle appearing at a road in direction $D$ at each timestep
       *     one idea is to make this a function of time
   * beta - prob. that a veh entering the system is a bicycle
   
 

<h1><center> Bike March </center></h1>
<center>Our first attempt moved every bike in the system on the same timestep (not realistic)</center>

<table>
<tr>
    <td> <img src="presfigs/bike_march-0.1.gif" alt="Bike March 0.1" style="width:500px"> </td>
    <td> <img src="presfigs/bike_march-0.5.gif" alt="Bike March 0.5" style="width:500px"> </td>
</tr>
</table>

<h3><center> Vermont State Bike Proportion of Bicycle Commuters = 0.006 </center></h3>
        <img src="trafsim_t-500_br-3_flow-N0.9-S0.1-E0.2-W0.3/figs/traffic_model-b2c-0006.gif" alt="Vermont" style="width:1000px">

<h3><center> Davis, CA Bike Proportion of Bicycle Commuters = 0.166 </center></h3>
<img src="trafsim_t-500_br-3_flow-N0.9-S0.1-E0.2-W0.3/figs/traffic_model-b2c-0166.gif" alt="San Francisco" style="width:1000px"> 

<h1><center> Busy Urban Streets </center></h1>

<table>
<tr>
    <td> <center> No Bikes -- beta = 0.00 </center> </td>
    <td> <center> SF, CA Bike Prop. -- beta = 0.039 </center> </td>
</tr>
<tr>
    <td> <img src="trafsim_t-500_br-3_flow-N0.4-S0.4-E0.4-W0.4/figs/traffic_model-b2c-0000.gif" alt="No Bikes" style="width:700px"> </td>
    <td> <img src="trafsim_t-500_br-3_flow-N0.4-S0.4-E0.4-W0.4/figs/traffic_model-b2c-0039.gif" alt="SF" style="width:700px"> </td>
</tr> 
<tr>
    <td> <center> Davis, CA Bike Prop. -- beta = 0.166 </center> </td>
    <td> <center> Copenhagen Bike Prop. -- beta = 0.57 </center> </td>
</tr>
<tr>
    <td> <img src="trafsim_t-500_br-3_flow-N0.4-S0.4-E0.4-W0.4/figs/traffic_model-b2c-0166.gif" alt="Davis, CA" style="width:700px"> </td>
    <td> <img src="trafsim_t-500_br-3_flow-N0.4-S0.4-E0.4-W0.4/figs/traffic_model-b2c-0460.gif" alt="Copenhagen" style="width:700px"> </td>
</tr>
</table>

<table>
<tr>
    <td> <center> No Bikes -- beta=0.00 </center> </td>
    <td> <center> Vermont Bike Prop. -- beta=0.006 </center> </td>
</tr>
<tr>
    <td> <img src="trafsim_t-500_br-3_flow-N0.4-S0.4-E0.4-W0.4/figs/beta_nobikes.png" alt="No Bikes" style="width:700px"> </td>
    <td> <img src="trafsim_t-500_br-3_flow-N0.4-S0.4-E0.4-W0.4/figs/beta_vt.png" alt="Vermont" style="width:700px"> </td>
</tr>
    <tr>
    <td> <center> SF, CA Bike Prop. -- beta = 0.039 </center> </td>
    <td> <center> Davis, CA Bike Prop. -- beta = 0.166 </center> </td>
</tr>
<tr>
    <td> <img src="trafsim_t-500_br-3_flow-N0.4-S0.4-E0.4-W0.4/figs/beta_sfCA.png" alt="SF" style="width:700px"> </td>
    <td> <img src="trafsim_t-500_br-3_flow-N0.4-S0.4-E0.4-W0.4/figs/beta_davisCA.png" alt="Davis" style="width:700px"> </td>
</tr>
    <tr>
    <td> <center> Copenhagen Bike Prop. -- beta = 0.46 </center> </td>
    <td> <center> Amsterdam Bike Prop. -- beta = 0.57 </center> </td>
</tr>
<tr>
    <td> <img src="trafsim_t-500_br-3_flow-N0.4-S0.4-E0.4-W0.4/figs/beta_copenhagen.png" alt="Copenhagen" style="width:700px"> </td>
    <td> <img src="trafsim_t-500_br-3_flow-N0.4-S0.4-E0.4-W0.4/figs/beta_amsterdam.png" alt="Amsterdam" style="width:700px"> </td>
</tr>
</table>

<h1><center> Possible Next Steps: </center></h1> 


  <center>1. More types of intersections: 2-way (L), 3-way </center>
  <center>2. Time dependent variables, such as flow rates</center>
  <center>3. 4-lane roads?</center>
  <center>4. Measure frequency and size of pile-ups</center>


In [ ]:
...